In [1]:
import mph

In [4]:
client = mph.start()
model = client.load("data/bend/beam.mph")

In [5]:
model.studies()

['Study 2']

In [7]:
model.solve("Study 2")

None


In [14]:
root = folder/'multiphysics'
architectures = {                      # valid system architecture names
    'Windows': ['win64'],
    'Linux':   ['glnxa64'],
    'Darwin':  ['maci64'],
}

In [17]:
for arch in architectures['Darwin']:
            comsol = root/'bin'/arch/'comsol'
            if comsol.is_file():
                print("here")
                break

In [18]:
comsol

PosixPath('/Applications/COMSOL61/multiphysics/bin/maci64/comsol')